# CIA World Factbook Data Analysis

## Introduction

In this project I'll be exploring the CIA World Factbook database which is a compilation of country statistics such as population and population growth. I will then analyse the data to uncover and extract valuable insights about countries worldwide based on some common metrics. I'll start by connecting my Jupyter notebook to the CIA World Factbook database.

In [3]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

## Data Overview

I can use the below query to return information from the database such as column info. This will help me to see what kind of data is contained in the dataset. I will limit the query results to the first ten rows of the 'Facts' table by using LIMIT.

In [2]:
%%sql
SELECT *
  FROM facts
 LIMIT 10;  

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46
6,ac,Antigua and Barbuda,442,442,0,92436,1.24,15.85,5.69,2.21
7,ar,Argentina,2780400,2736690,43710,43431886,0.93,16.64,7.33,0.0
8,am,Armenia,29743,28203,1540,3056382,0.15,13.61,9.34,5.8
9,as,Australia,7741220,7682300,58920,22751014,1.07,12.15,7.14,5.65
10,au,Austria,83871,82445,1426,8665550,0.55,9.41,9.42,5.56


Columns such as area. area_land, and area_water are presented in square kilometres. Population_growth is calculated on a yearly basis by calculating the average annual percent change in population. Birth_rate, death_rate, and migration_rate is calculated by taking the average annual number during a given year per 1,000 persons.

## Analysing Data using Functions

I can start my analysis by using SQL aggregate functions such as MIN which returns the smallest value and MAX which returns the largest value.
Population and population growth will be the first columns that I will query information from.

In [4]:
%%sql
SELECT MIN(population) AS min_population,
       MAX(population) AS max_population,
       MIN(population_growth) AS min_population_growth,
       MAX(population_growth) AS max_population_growth
  FROM facts; 

Done.


min_population,max_population,min_population_growth,max_population_growth
0,7256490011,0.0,4.02


Some interesting points that stand out to me from this query:
-  There is a country with a minimum population and population growth of 0. This is unusual as all countries have some sort of indigenous population.
-  There is a country with a population of 7,256,490,011 which is around 7.2 billion. This is several billion more than what I think the most populous country is.
I have an idea of what this can be and will explain this further in the next section.

## Exploring the Population Data

I can determine the minimum and maximum populations by using the queries below. The HAVING clause will be used since WHERE can't be used with aggregate functions like MIN and MAX.'

In [5]:
%%sql
SELECT MIN(population), name
  FROM facts
 GROUP BY name
HAVING MIN(population) = 0;

Done.


MIN(population),name
0,Antarctica


This matches the country information for Antarctica if I search it up on the CIA website. The description for Antarctica states that there are no indigenous inhabitants on the continent but there are personnel such as scientists and ship crew that reside in the summer and winter months. Furthermore, the population fluctuates from around 4,400 in the summer months and 1,100 in the winter months.

In [2]:
%%sql
SELECT MAX(population), name
  FROM facts
 GROUP BY name
HAVING MAX(population) = 7256490011;

Done.


MAX(population),name
7256490011,World


After running these queries I can see that the country with a minimum population of 0 is Antarctica and the country with a maximum population of 7,256,490,011 is World. The dataset that we are querying information from uses regions and countries which distorts the results that we queried. In order to remedy this I can determine the most populous country by excluding the name "World" from the results. This can be achieved with the equality operator "!=" in SQL. Depending on the SQL program, "!=" or 
"<>" can be used to filter out this value.

In [14]:
%%sql
SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) AS max_pop_growth 
  FROM facts
 WHERE name != 'World';

Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,1367485388,0.0,4.02


I now know that the most populous country should have a population of 1,367,485,388 (around 1.3 billion). This can be determined by filtering out the countries with a population of 0 and ordering the countries by the largest populations. The top 10 names will only be displayed.

In [16]:
%%sql
SELECT name, population
  FROM facts
 WHERE population != 0
ORDER BY population DESC
LIMIT 10;

Done.


name,population
World,7256490011
China,1367485388
India,1251695584
European Union,513949445
United States,321368864
Indonesia,255993674
Brazil,204259812
Pakistan,199085847
Nigeria,181562056
Bangladesh,168957745


This query confirms that China is the most populous country and also tells us that regions such as European Union are included in the results. I can exclude the European Union and World from the top 10 results by filtering it out in the WHERE clause below by using NOT IN and ordering the population by descending order.

In [6]:
%%sql
SELECT name, population
  FROM facts
 WHERE name NOT IN ('World', 'European Union')  
 ORDER BY population DESC
 LIMIT 10;   

Done.


name,population
China,1367485388
India,1251695584
United States,321368864
Indonesia,255993674
Brazil,204259812
Pakistan,199085847
Nigeria,181562056
Bangladesh,168957745
Russia,142423773
Japan,126919659


By using the above query we can only include countries in the top 10 most populous countries and not regions.